In [1]:
import sys, os

try:
    # ✅ Running from a Python script (.py file)
    base_path = os.path.abspath(os.path.join(os.path.dirname(__file__), "..", ".."))
except NameError:
    # ✅ Running from a Jupyter notebook (__file__ is not defined)
    base_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

SRC_PATH = os.path.join(base_path)

if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print(f"✅ SRC path added: {SRC_PATH}")
else:
    print(f"🔁 SRC path already in sys.path: {SRC_PATH}")

✅ SRC path added: /home/prashant-agrawal/projects/netflix_talk2data/src


In [2]:
# 🚀 Import your utility loaders
from utils.qdrant_client_loader import get_qdrant_collection_name


# 📂 Define paths and configurations

COLLECTION_NAME = get_qdrant_collection_name()

print(f"📌 Collection Name: {COLLECTION_NAME}")


📌 Collection Name: indian_startups


In [3]:
# --- Utility: Normalization ---
def normalize_field_name(field: str) -> str:
    return (
        field.strip().lower()
        .replace(" ", "_").replace("(", "").replace(")", "")
        .replace("/", "_")
    )

def normalize_field_value(value) -> str:
    return str(value).strip().lower()

In [4]:
# src/tools/qdrant_tool.py

import re
from typing import List, Dict, Any, Union

from qdrant_client import QdrantClient
from qdrant_client.http.models import FieldCondition, MatchValue, Range, Filter
from langchain_openai import OpenAIEmbeddings


class QdrantSearchTool:
    """
    Tool for performing hybrid semantic + metadata searches against a Qdrant collection.
    """

    def __init__(
        self,
        host: str,
        port: int,
        collection_name: str,
        embedding_model: OpenAIEmbeddings,
    ):
        self.client = QdrantClient(host=host, port=port)
        self.collection = collection_name
        self.embedding_model = embedding_model

    @staticmethod
    def _normalize_field_name(field: str) -> str:
        f = field.strip().lower()
        f = re.sub(r"[ ()/]", "_", f)
        return re.sub(r"[^a-z0-9_]", "", f)

    @staticmethod
    def _normalize_field_value(value: Any) -> str:
        return str(value).strip().lower()

    def _build_filter(self, filters: Dict[str, Union[str, int, float, Dict[str, Any]]]) -> Filter:
        """
        Convert a user-provided dict of filters into a Qdrant Filter object.
        Supports:
          - exact match: {"state": "delhi"}
          - range match: {"year_founded": {"gte": 2000, "lte": 2010}}
        """
        conditions = []
        for raw_field, cond in filters.items():
            key = self._normalize_field_name(raw_field)

            if isinstance(cond, dict) and ("gte" in cond or "lte" in cond):
                conditions.append(
                    FieldCondition(
                        key=key,
                        range=Range(gte=cond.get("gte"), lte=cond.get("lte")),
                    )
                )
            else:
                val = self._normalize_field_value(cond)
                conditions.append(
                    FieldCondition(key=key, match=MatchValue(value=val))
                )

        return Filter(must=conditions)

    def search(
        self,
        query: str,
        filters: Dict[str, Union[str, int, float, Dict[str, Any]]] = None,
        k: int = 5,
    ) -> List[Dict[str, Any]]:
        """
        Perform a similarity search with optional metadata filtering.
        Returns a list of dicts: { "id", "score", "payload" }.
        """
        # 1. Embed the query
        vector = self.embedding_model.embed_query(query)

        # 2. Build Qdrant filter if provided
        q_filter = self._build_filter(filters) if filters else None

        # 3. Execute search
        results = self.client.search(
            collection_name=self.collection,
            query_vector=vector,
            query_filter=q_filter,
            limit=k,
            with_payload=True,
        )

        # 4. Format output
        output = []
        for pt in results:
            output.append({
                "id": pt.id,
                "score": pt.score,
                "payload": pt.payload,
            })
        return output

# 4️⃣ Instantiate once
embedding_model = OpenAIEmbeddings()
qdrant_search_tool = QdrantSearchTool(
    host="localhost",
    port=6333,
    collection_name=COLLECTION_NAME,
    embedding_model=embedding_model
)

In [5]:

# Wrapper for LangChain agent compatibility
def wrapped_qdrant_search(inputs: dict) -> list:
    query = inputs.get("query", "")
    filters = inputs.get("filters", None)
    k = inputs.get("k", 5)
    print(f"\n[DEBUG] Query: {query}")
    print(f"[DEBUG] Filters: {filters}")
    print(f"[DEBUG] Top K: {k}")
    try:
        results = qdrant_search_tool.search(query=query, filters=filters, k=k)
        print(f"[DEBUG] Raw results: {results}")
        return results
    except Exception as e:
        print(f"[ERROR] Qdrant search failed: {e}")
        return []

In [6]:
# 5️⃣ Test functions
def test_semantic():
    print("🔍 Test: pure semantic (no filters)")
    for r in qdrant_search_tool.search(query="emerging fintech startups", k=3):
        print(f" • [{r['score']:.4f}] {r['payload'].get('company_name')}")

def test_metadata():
    print("🔍 Test: metadata-only filter state=delhi")
    for r in qdrant_search_tool.search(query="", filters={"state": "delhi"}, k=5):
        print(f" • {r['payload']['company_name']} (state={r['payload']['state']})")

def test_range():
    print("🔍 Test: range filter year_founded in [2000,2010]")
    for r in qdrant_search_tool.search(
        query="",
        filters={"year_founded": {"gte": 2000, "lte": 2010}},
        k=5
    ):
        print(f" • {r['payload']['company_name']} (founded={r['payload']['year_founded']})")

# 6️⃣ Run all tests
test_semantic()
print()
test_metadata()
print()
test_range()


🔍 Test: pure semantic (no filters)


/tmp/ipykernel_7014/792566242.py:80: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = self.client.search(


 • [0.7726] boat
 • [0.7726] None
 • [0.7726] None

🔍 Test: metadata-only filter state=delhi
 • cred (state=delhi)
 • curefit (state=delhi)
 • tork motors (state=delhi)
 • lenskart (state=delhi)
 • yulu (state=delhi)

🔍 Test: range filter year_founded in [2000,2010]
